In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely import wkt
import plotly.express as px

In [2]:
data = gpd.read_file("Etablissements_santé.geojson")
data.head()

,type,name,operator,emergency,wheelchair,opening_hours,capacity,for_public,phone,ref_finess,...,meta_code_dep,meta_name_reg,meta_code_reg,meta_osm_id,meta_osm_url,meta_first_update,meta_last_update,meta_versions_number,meta_users_number,geometry
0,nursing_home,Ehpad du CH,None,None,None,None,None,senior,+33385884493,710013509,...,71,Bourgogne-Franche-Comté,27,7696263271,https://www.openstreetmap.org/node/7696263271,2020-07-08 21:24:22+00:00,2022-08-18,2.0,2.0,POINT (4.1241 46.45343)
1,nursing_home,Ehpad du CH,None,None,None,None,None,senior,+33474065250,690787510,...,69,Auvergne-Rhône-Alpes,84,7701544666,https://www.openstreetmap.org/node/7701544666,2020-07-10 07:47:21+00:00,2022-08-18,2.0,2.0,POINT (4.74738 46.11017)
2,nursing_home,Ehpad du CH,None,None,None,None,None,senior,+33385796900,710972977,...,71,Bourgogne-Franche-Comté,27,7701680916,https://www.openstreetmap.org/node/7701680916,2020-07-10 08:43:03+00:00,2022-08-18,2.0,2.0,POINT (4.13717 46.6923)
3,dentist,Dr Loudmilla Samya Oukaci,None,None,None,None,None,None,None,None,...,57,Grand Est,44,7722363124,https://www.openstreetmap.org/node/7722363124,2020-07-16 23:15:36+00:00,2023-08-28,2.0,1.0,POINT (6.17879 49.09646)
4,nursing_home,EHPAD Les Orchidées,SAS les Orchidées,None,None,None,27,senior,+33 4 93 40 54 64,060799020,...,06,Provence-Alpes-Côte d'Azur,93,7723453687,https://www.openstreetmap.org/node/7723453687,2020-07-17 07:46:50+00:00,2022-08-18,2.0,2.0,POINT (6.92928 43.65897)


In [3]:
# Listes des codes INSEE de la métropole de Grenoble et de Rennes
codes_grenoble = [
    "38057","38059","38068","38071","38111","38126","38150","38151","38158",
    "38169","38170","38179","38185","38187","38188","38200","38229","38235",
    "38252","38258","38271","38277","38279","38281","38309","38317","38325",
    "38328","38364","38382","38388","38421","38423","38436","38445","38471",
    "38472","38474","38478","38485","38486","38516","38524","38528","38529",
    "38533","38540","38545","38562"
]
codes_rennes = [
    "35001","35022","35024","35032","35039","35047","35051","35055","35058",
    "35059","35065","35066","35076","35079","35080","35081","35088","35120",
    "35131","35139","35144","35180","35189","35196","35204","35206","35208",
    "35210","35216","35238","35240","35245","35250","35266","35285","35288",
    "35315","35334","35351","35352","35353","35355","35363"
]
data_filtre = data[data["meta_code_com"].isin(codes_grenoble + codes_rennes)]

In [4]:
colonnes_a_supprimer = [
    'operator', 'emergency', 'wheelchair', 'opening_hours',
    'capacity', 'for_public', 'phone', 'ref_finess', 'type_finess',
    'ref_naf', 'ref_siret', 'meta_osm_id', 'meta_osm_url',
    'meta_first_update', 'meta_last_update',
    'meta_versions_number', 'meta_users_number'
]
data_filtre = data_filtre.drop(columns=colonnes_a_supprimer)

In [5]:

data_filtre = data_filtre.rename(columns={
    "type": "type_etablissement",
    "name": "nom_etablissement",
    "meta_name_com": "commune",
    "meta_code_com": "code_commune",
    "meta_name_dep": "departement",
    "meta_code_dep": "code_departement",
    "meta_name_reg": "region",
    "meta_code_reg": "code_region",
})
data_filtre["Métropole"] = np.where(
    data_filtre["code_commune"].isin(codes_grenoble),
    "Grenoble",
    "Rennes"
)
data_filtre.head()

,type_etablissement,nom_etablissement,commune,code_commune,departement,code_departement,region,code_region,geometry,Métropole
47,dentist,Olivier Quéré,Rennes,35238,Ille-et-Vilaine,35,Bretagne,53,POINT (-1.6634 48.09456),Rennes
65,hospital,Hôpital de jour Addictologie,Saint-Martin-d'Hères,38421,Isère,38,Auvergne-Rhône-Alpes,84,POINT (5.75351 45.185),Grenoble
86,pharmacy,Pharmacie Jaouen,Vezin-le-Coquet,35353,Ille-et-Vilaine,35,Bretagne,53,POINT (-1.75734 48.11907),Rennes
99,pharmacy,Pharmacie Grangeon,Échirolles,38151,Isère,38,Auvergne-Rhône-Alpes,84,POINT (5.71883 45.15646),Grenoble
166,doctors,Podologue pédicure,Grenoble,38185,Isère,38,Auvergne-Rhône-Alpes,84,POINT (5.74132 45.17814),Grenoble


In [6]:
gdf = gpd.GeoDataFrame(data_filtre, geometry="geometry", crs="EPSG:4326")
gdf["lon"] = gdf.geometry.x
gdf["lat"] = gdf.geometry.y
grenoble = gdf[gdf["Métropole"] == "Grenoble"]
fig_map = px.scatter_mapbox(
    grenoble,
    lat="lat",
    lon="lon",
    hover_name="nom_etablissement",
    hover_data={
        "type_etablissement": True,
        "commune": True,
        "nom_etablissement": False
    },
    color="type_etablissement",
    zoom=10,
    title="Établissements de santé - Métropole de Grenoble",
    height=600
)
fig_map.update_layout(
    mapbox_style="carto-positron",  # 👈 fond noir & blanc
    template="plotly_white",
    margin={"r": 0, "t": 50, "l": 0, "b": 0}
)
fig_map.show()

/var/folders/_1/1lx811qj7f19ynqlp2p83tcw0000gn/T/ipykernel_13094/341224695.py:5: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig_map = px.scatter_mapbox(


In [7]:
data_filtre.to_file("Etablissements_santé_filtre.geojson", driver="GeoJSON")

In [8]:
election_1er_tour = pd.read_excel("Election_1er_tour.xlsx")
election_1er_tour.head()

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Code B.Vote,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,% Voix/Exp.60,N.Pan..61,Code Nuance.61,Sexe.61,Nom.61,Prénom.61,Liste.61,Voix.61,% Voix/Ins.61,% Voix/Exp.61
0,1,Ain,1,L'Abergement-Clémenciat,1,622,340,54.66,282,45.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Ain,2,L'Abergement-de-Varey,1,212,83,39.15,129,60.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Ain,4,Ambérieu-en-Bugey,1,1042,686,65.83,356,34.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Ain,4,Ambérieu-en-Bugey,2,1064,722,67.86,342,32.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Ain,4,Ambérieu-en-Bugey,3,1120,696,62.14,424,37.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
departements = ["Ille-et-Vilaine", "Isère"]
communes_grenoble = [
    "Bresson","Brié-et-Angonnes","Champ-sur-Drac","Champagnier","Claix",
    "Corenc","Domène","Echirolles","Eybens","Fontaine","Fontanil-Cornillon",
    "Gières","Grenoble","Herbeys","Jarrie","La Tronche","Le Gua",
    "Le Pont-de-Claix","Le Sappey-en-Chartreuse","Meylan","Miribel-Lanchâtre",
    "Mont-Saint-Martin","Montchaboud","Murianette","Notre-Dame-de-Commiers",
    "Notre-Dame-de-Mésage","Noyarey","Poisat","Proveysieux","Quaix-en-Chartreuse",
    "Saint-Barthélemy-de-Séchilienne","Saint-Egrève","Saint-Georges-de-Commiers",
    "Saint-Martin-d'Hères","Saint-Martin-le-Vinoux","Saint-Paul-de-Varces",
    "Saint-Pierre-de-Mésage","Sarcenas","Sassenage","Séchilienne",
    "Seyssinet-Pariset","Seyssins","Varces-Allières-et-Risset",
    "Vaulnaveys-le-Bas","Vaulnaveys-le-Haut","Venon","Veurey-Voroize",
    "Vif","Vizille"
]
communes_rennes = [
    "Acigné", "Bécherel","Betton","Bourgbarré","Brécé","Bruz","Cesson-Sévigné",
    "Chantepie","Chartres-de-Bretagne","Chavagne","Chevaigné","Cintré","Corps-Nuds","Gévezé",
    "La Chapelle-des-Fougeretz","La Chapelle-Thouarault","L'Hermitage","Le Rheu","Le Verger",
    "Montgermont","Mordelles","Noyal-Châtillon-sur-Seiche","Nouvoitou","Orgères",
    "Pacé","Parthenay-de-Bretagne","Pont-Péan","Rennes","Romillé","Saint-Armel","Saint-Erblon",
    "Saint-Gilles","Saint-Grégoire","Saint-Jacques-de-la-Lande","Saint-Sulpice-la-Forêt",
    "Thorigné-Fouillard","Vern-sur-Seiche","Vezin-le-Coquet", "Clayes", "La Chapelle-Chaussée", 
    "Laillé", "Langan", "Miniac-sous-Bécherel"
] 

election_1er_tour_filtre = election_1er_tour[
    (election_1er_tour["Libellé du département"].isin(departements)) &
    (election_1er_tour["Libellé de la commune"].isin(communes_grenoble + communes_rennes))
]
# Récupérer l'index de la colonne "Sexe"
idx_sexe = election_1er_tour_filtre.columns.get_loc("Sexe")
# Garder uniquement les colonnes jusqu'à "Sexe" inclus
election_1er_tour_filtre = election_1er_tour_filtre.iloc[:, :idx_sexe+1]
election_1er_tour_filtre.to_csv("election_1er_tour_filtre.csv", index=False)


In [10]:
election_2eme_tour = pd.read_excel("Election_2eme_tour.xlsx")
election_2eme_tour.head()

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Code B.Vote,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,Unnamed: 558,Unnamed: 559,Unnamed: 560,Unnamed: 561,Unnamed: 562,Unnamed: 563,Unnamed: 564,Unnamed: 565,Unnamed: 566,Unnamed: 567
0,01,Ain,12,Aranc,0001,273,120,43.96,153,56.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Ain,34,Belley,0001,892,422,47.31,470,52.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Ain,34,Belley,0002,973,471,48.41,502,51.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,Ain,34,Belley,0003,967,546,56.46,421,43.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,Ain,34,Belley,0004,969,491,50.67,478,49.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
election_2eme_tour_filtre = election_2eme_tour[
    (election_2eme_tour["Libellé du département"].isin(departements)) &
    (election_2eme_tour["Libellé de la commune"].isin(communes_grenoble + communes_rennes))
]
# Récupérer l'index de la colonne "Sexe"
idx_sexe = election_2eme_tour_filtre.columns.get_loc("Sexe")
# Garder uniquement les colonnes jusqu'à "Sexe" inclus
election_2eme_tour_filtre = election_2eme_tour_filtre.iloc[:, :idx_sexe+1]
election_2eme_tour_filtre.to_csv("election_2eme_tour_filtre.csv", index=False)